# Module

In [12]:
import warnings
warnings.filterwarnings(action='ignore')

import json
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
import requests
import json
from glob import glob
import time
import pandas as pd
from tqdm import tqdm
from urllib.parse import unquote
from collections import defaultdict
from bs4 import BeautifulSoup

from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from PIL import Image, ImageDraw

# HTML 
* 하나의 PDF로 부터 나온 html tag는, id값이 1씩 increase 하도록 후처리
* 추후 해당 id를 활용해, json으로 부터 dounding_box와 page를 가져올 예정

In [45]:
# html 파일을 읽어서, id 값을 수정하는 함수
def change_html_tag_id(html_path, save_path):
    html = open(html_path).read()
    soup = BeautifulSoup(html, 'html.parser')
    re = 0
    for tag in soup.find_all():
        if tag.get('id'):
            tag['id'] = re
            re += 1
    with open(save_path, 'w') as f:    
        f.write(str(soup))  


In [51]:
html_paths = glob("../data/nursing/nursing_html/*")
for html_path in html_paths:
    file_name = os.path.basename(html_path)
    change_html_tag_id(html_path, f"../data/nursing/nursing_html2/{file_name}")

# JSON
* Document Parser로 부터 저장된 JSON 파일을 PDF 단위로 결합
* 결합시, id는 0부터 시작하여 HTML의 파일과 동일하게 매칭

In [2]:
def json_post_process(file_paths, save_folder_path):
    file_names = list(set([os.path.basename(file_path).split(".pdf")[0] for file_path in file_paths]))
    re = defaultdict(list)

    # 파일 이름이 같은 것들을 묶어준다.
    for file_name in file_names:
        for file_path in file_paths:
            if file_name in file_path:
                re[file_name].append(file_path)
    
    # 정렬 해 준다.
    for file_name in re:
        re[file_name] = sorted(re[file_name])
    
    for file_name in re:
        re_json = {"elements":[]}
        for file_path in re[file_name]:
            with open(file_path, "r") as f:
                data = json.load(f)
                re_json["elements"].extend(data["elements"])
        id_ = 0
        for r in re_json["elements"]:
            r["id"] = id_
            id_ += 1
            for b in r["bounding_box"]:
                b["x"] = b["x"]/1362
                b["y"] = b["y"]/1776
        
        with open(os.path.join(save_folder_path, file_name+".json"), 'w') as f:
            json.dump(re_json, f, indent=4)

In [3]:
json_post_process(glob("../data/nursing/nursing_downloads/*.json"), "../data/nursing/nursing_json")

# PDF to PNG
* PDF 파일을 PNG 이미지로 변환
* 형식 : 파일이름_page

In [13]:
pdf_paths = glob("../data/nursing/*.pdf")

In [14]:
pdf_paths

['../data/nursing/22장 신경계장애 대상자간호-(1).pdf',
 '../data/nursing/18장 요로계 장애 대상자 간호.pdf',
 '../data/nursing/14장 호흡기장애 대상자 간호-(2).pdf',
 '../data/nursing/17장 혈액계 장애 대상자 간호.pdf',
 '../data/nursing/21장 근골격계 장애.pdf',
 '../data/nursing/20장 유방장애 대상자 간호.pdf',
 '../data/nursing/27장 화상간호.pdf',
 '../data/nursing/25장 청각과 평형장애 대상자 간호.pdf',
 '../data/nursing/30 판권.pdf',
 '../data/nursing/26장 피부장애 대상자 간호.pdf',
 '../data/nursing/16장 혈관장애 대상자 간호.pdf',
 '../data/nursing/19장 남성 생식기계 대상자 간호.pdf',
 '../data/nursing/14장 호흡기장애 대상자 간호-(1).pdf',
 '../data/nursing/22장 신경계장애 대상자간호-(2).pdf',
 '../data/nursing/00 도입부(하권).pdf',
 '../data/nursing/23장 내분비장애 대상자 간호.pdf',
 '../data/nursing/24장 시각장애 대상자 간호.pdf',
 '../data/nursing/15장 심장장애 대상자 간호-(2).pdf',
 '../data/n

In [16]:
for pdf_path in tqdm(pdf_paths):
    images = convert_from_path(pdf_path, dpi=150)
    file_name = os.path.basename(pdf_path).split(".pdf")[0]
    
    if not os.path.exists(f"../data/nursing/nursing_images/{file_name}"):
        os.makedirs(f"../data/nursing/nursing_images/{file_name}")

    for i, image in enumerate(images):
        image.save(f"../data/nursing/nursing_images/{file_name}/{i}.png")

100%|██████████| 19/19 [09:26<00:00, 29.82s/it]


In [ ]:
images = convert_from_path("/home/livin/rag_pipeline/AutoRAG/data/nursing/14장 호흡기장애 대상자 간호-(1).pdf", dpi=150)